# Gold Layer Fact Table

## Load Taxi Data from Silver Zone

In [1]:
df_silvertaxi = spark.read.format("delta").load("abfss://f16d7a5a-40df-480f-9cfe-26f14dedc25a@onelake.dfs.fabric.microsoft.com/d8e1915f-0286-4a2b-9c7d-f9b0cd8a5167/Tables/silver_nyctlcYellow")

silverRowCount = "{:,}".format(df_silvertaxi.count())
print(silverRowCount)

StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 3, Finished, Available)

1,571,671,152


## Create temporary view to work with Spark SQL

In [2]:

df_silvertaxi.createOrReplaceTempView("NYCTaxiYellow_Silver")


StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 4, Finished, Available)

## Create Temporary view that transforms the data 

In [3]:
%%sql


CREATE OR REPLACE TEMP VIEW TNYCTaxiYellow AS 
SELECT 

IF (t.PassengerCount = 1,1,2) as PassengerCategoryKey, 
abs((unix_timestamp(tpepDropoffDateTime) - unix_timestamp(tpepPickupDateTime)))/60 as DurationMinutes, 

If (floor((abs((unix_timestamp(tpepDropoffDateTime) - unix_timestamp(tpepPickupDateTime)))/60)/15) > 8, 8 ,
    floor((abs((unix_timestamp(tpepDropoffDateTime) - unix_timestamp(tpepPickupDateTime)))/60)/15)) as DurationBucketKey, 


EXTRACT( year from tpepPickupDateTime ) * 10000 + 
    EXTRACT( month from tpepPickupDateTime) * 100 + 
        EXTRACT( day from tpepPickupDateTime) as tpepPickupDateKey, 

CASE WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=259 THEN 0
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=659 THEN 1
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=859 THEN 2
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=1159 THEN 3
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=1359 THEN 4
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=1559 THEN 5
    WHEN (hour(tpepPickupDateTime) * 100 + minute(tpepPickupDateTime)) <=1759 THEN 6
    ELSE 7 END as DayTimeBucketGroupKey, 

    t.* 
    from NYCTaxiYEllow_Silver t
    where t.vendorID in ( 1,2,3,4)

StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 5, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

## Load transformed Fact data to a dataframe

In [4]:
df_FactNycTaxiYellow = spark.sql("SELECT * FROM TNYCTaxiYellow")
goldRowCount =  "{:,}".format(df_FactNycTaxiYellow.count())
print(goldRowCount)

StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 6, Finished, Available)

538,038,292


In [5]:
display(df_FactNycTaxiYellow)

StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 18528885-e8ff-4aea-a8aa-0bca493b1b2f)

## write the fact table

In [6]:
df_FactNycTaxiYellow.write \
.mode("overwrite").format("delta").partitionBy("puYear") \
.option("overwriteSchema", "true") \
.save("Tables/" + "FactNycTaxiYellow")

StatementMeta(, 94201c01-5b71-473c-9539-bebc478a52d1, 8, Submitted, Running)